See [Scala and jupyter notebook](https://jonnylaw.rocks/posts/2019-04-15-scala-and-jupyter-notebook-with-almond/) for some info about setup.

In [1]:
import $ivy.`com.beachape::enumeratum:1.7.3`, enumeratum.{Enum, EnumEntry}
import $ivy.`com.eed3si9n.expecty::expecty:0.16.0`, com.eed3si9n.expecty.Expecty.expect
import fastparse._

import $ivy.$                               , enumeratum.{Enum, EnumEntry}

import $ivy.$                                     , com.eed3si9n.expecty.Expecty.expect

import fastparse._

In [2]:
// Define the set of built-in symbols supported in µDhall.

sealed abstract class Constant(override val entryName: String) extends EnumEntry {}

object Constant extends Enum[Constant] {
  override def values = findValues

  case object Natural         extends Constant("Natural")
  case object NaturalFold     extends Constant("Natural/fold")
  case object NaturalIsZero   extends Constant("Natural/isZero")
  case object NaturalSubtract extends Constant("Natural/subtract")
  case object Kind            extends Constant("Kind")
  case object Type            extends Constant("Type")
}

defined class Constant
defined object Constant

In [3]:
// Define the set of built-in binary operators supported in µDhall.

sealed abstract class Operator(val name: String, val precedence: Int) extends EnumEntry

object Operator extends Enum[Operator] {
  val values = findValues
  // These operators work only with values of type Natural.
  case object Plus extends Operator("+", 20)
  case object Times extends Operator("*", 10)
}

defined class Operator
defined object Operator

In [4]:
sealed trait Expr

object Expr {
  // Natural literals, for example 123
  final case class NaturalLiteral(value: Int) extends Expr {
      require(value >= 0)
  }
  // Variables with their de Bruijn indices.
  final case class Variable(name: String, index: Int = 0) extends Expr {
      require (index >= 0)
  }
  // λ(name : tpe) → body  -- Function literal value.
  final case class Lambda(name: String, tpe: Expr, body: Expr) extends Expr

  // ∀(name : tpe) → body  -- Function type.
  final case class Forall(name: String, tpe: Expr, body: Expr) extends Expr

  // let name = subst in body  -- Locally scoped variable definition.
  final case class Let(name: String, subst: Expr, body: Expr) extends Expr

  // body : tpe   -- Type annotation.
  final case class Annotated(body: Expr, tpe: Expr) extends Expr

  // func arg   -- Application of a function to an argument.
  final case class Applied(func: Expr, arg: Expr) extends Expr

  // Built-in constant symbols such as "Natural" or "Type".
  final case class Builtin(constant: Constant) extends Expr

  // Binary operations such as "n + 123".
  final case class BinaryOp(left: Expr, op: Operator, right: Expr) extends Expr
}

cmd4.sc:182: The outer reference in this type test cannot be checked at run time.
  final case class NaturalLiteral(value: Int) extends Expr {
                   ^
cmd4.sc:186: The outer reference in this type test cannot be checked at run time.
  final case class Variable(name: String, index: Int = 0) extends Expr {
                   ^
cmd4.sc:190: The outer reference in this type test cannot be checked at run time.
  final case class Lambda(name: String, tpe: Expr, body: Expr) extends Expr
                   ^
cmd4.sc:193: The outer reference in this type test cannot be checked at run time.
  final case class Forall(name: String, tpe: Expr, body: Expr) extends Expr
                   ^
cmd4.sc:196: The outer reference in this type test cannot be checked at run time.
  final case class Let(name: String, subst: Expr, body: Expr) extends Expr
                   ^
cmd4.sc:199: The outer reference in this type test cannot be checked at run time.
  final case class Annotated(body: Expr, t

defined trait Expr
defined object Expr

In [5]:
val test = Expr.BinaryOp(Expr.NaturalLiteral(123), Operator.Plus, Expr.Variable("a", 0))

test: Expr.BinaryOp = BinaryOp(
  left = NaturalLiteral(value = 123),
  op = Plus,
  right = Variable(name = "a", index = 0)
)

In [6]:
object DSL { // Helper methods for creating µDhall values more easily in Scala.
    import Expr._
    import Constant._
    import Operator._
    
    implicit class IntroduceVar(name: String) {
        def ! : Variable = Variable(name)
        def !!(index: Int) : Variable = Variable(name, index)
    }
    implicit class IntroduceNatural(n: Int) {
        def ! : NaturalLiteral = NaturalLiteral(n)
    }
    implicit class IntroduceSymbol(c: Constant) {
        def ! : Expr = Builtin(c)
    }
    implicit class NaturalOps(e: Expr) {
        def +(other: Expr): Expr = BinaryOp(e, Plus, other)
        def *(other: Expr): Expr = BinaryOp(e, Times, other)
    }
    implicit class ExprAnnotate(e: Expr) {
        def :~(tpe: Expr): Expr = Annotated(e, tpe)
        def apply(arg: Expr): Expr = Applied(e, arg)
        // Instead of "let x = e in body" we write body.let(x, e)
        def let(arg: String, subst: Expr): Expr = Let(arg, subst, e)
    }
    implicit class ExprFunc(x: Expr) {
        // Instead of "λ(name : tpe) → body" we write name.! :~ tpe ~> body
        def ~>(body: Expr): Expr = x match {
            case Annotated(Variable(v, 0), tpe) => Lambda(v, tpe, body)
            case _ => throw new Exception(s"Invalid Lambda: argument must be an Annotated name but instead got $x")
        }
        // Instead of "∀(name : tpe) → body" we write name.! :~ tpe :~> body
        def :~>(body: Expr): Expr = x match {
            case Annotated(Variable(v, 0), tpe) => Forall(v, tpe, body)
            case _ => throw new Exception(s"Invalid Forall: argument must be an Annotated name but instead got $x")
        }
    }
}

cmd6.sc:29: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tpe) => Lambda(v, tpe, body)
                          ^
cmd6.sc:29: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tpe) => Lambda(v, tpe, body)
                                   ^
cmd6.sc:34: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tpe) => Forall(v, tpe, body)
                          ^
cmd6.sc:34: The outer reference in this type test cannot be checked at run time.
            case Annotated(Variable(v, 0), tpe) => Forall(v, tpe, body)
                                   ^


defined object DSL

In [27]:
object Test1 {
    // A simple test.
    import DSL._
    import Constant._

    val test0 = 1.! + 2.! + 3.!
    val test1 = "n".! + 123.! :~  Natural.!
    val test2 = ("n".! :~ Natural.!) ~> ("n".! + 1.!)
    val test3 = ("n".! :~ Natural.!) :~> Natural.!
    val test4 = test2 :~ test3
    val test5 = test2(test1)
    val test6 = "f".! ("g".! ("x".!))
}

defined object Test1

In [32]:
/*
   The pretty-printer works by computing the "inner" and "outer" binding precedence of each expression.

   - Parentheses are required whenever the outer precedence is below the inner precedence.

   Some examples:

   a   *   b   +   c                 Plus( Times (a, b), c )
      10      20

   (a   +   b)   *   f     (g   c)           Times ( Plus (a, b), Applied(f, Applied(g, c) ) )
       20       10     5,4    5

   f   a     (b   +   c)   +   d       Plus( Applied ( Applied (f, a), Plus (b, c) ), d )
     5   5,4     20       20

   λ(a : Natural) → λ(b : Natural)  →  f   a   +   b    Lambda ( a, Natural, Lambda(b, Natural, Plus (Applied(f, a), b) ) )
       8          3     8           3    5    20

   (λ(a : Natural)  →  a)   b         Applied (Lambda (a, Natural, a), b)
        8           3     5  

   - Each of the Expr constructors has an overall outer precedence and a separate inner precedence for each Expr argument.

   - Precedence values must be specified separately for each constructor and each argument.

   Applied(f, a) has outer precedence 5 and inner precedence 

   - Binary operations have equal outer and inner precedence values. This is the "precedence of the operation".

   - Other constructors sometimes have unequal outer and inner precedence values.
*/

// Return (outer, List(inner1, inner2, ...)) for each constructor that may have Expr arguments.
def precedence(e: Expr): (Int, List[Int]) = e match {
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
    case Expr.Lambda(name, tipe, body) => (50, List(50, 50))
    case Expr.Forall(name, tipe, body) => (50, List(50, 50))
    case Expr.Let(name, subst, body) => (50, List(50, 50))
    case Expr.Annotated(body, tipe) => (8, List(7, 60))   //   ( 1 : Natural ) : Natural : Type
    case Expr.Applied(func, arg) => (5, List(4, 4))      //   f (g x)
    case Expr.BinaryOp(_, op, _) => (op.precedence, List(op.precedence, op.precedence)) 
}

def inPrecedence(expr: String, innerPrec: Int, outerPrec: Int): String =
    if (innerPrec > outerPrec) s"($expr)" else expr

def prettyprint(e: Expr, outside: Int = 100): String = {
  val (outer, inner) = precedence(e)

def printpair(prefix: String, left: Expr, middle: String, right: Expr): String =
    prefix + prettyprint(left, inner(0)) + middle + prettyprint(right, inner(1))

  val exprPrinted = e match {
    case Expr.NaturalLiteral(value) => value.toString
    case Expr.Variable(name, index) => name + (if (index != 0) s"@$index" else "")
    case Expr.Builtin(constant) => constant.entryName
    case Expr.Lambda(name, tipe, body) => printpair(s"λ($name : ", tipe, ") → ", body)
    case Expr.Forall(name, tipe, body) => printpair(s"∀($name : ", tipe, ") → ", body)
    case Expr.Let(name, subst, body) => printpair(s"let $name = ", subst, " in ", body)
    case Expr.Annotated(body, tipe) => printpair("", body, " : ", tipe)
    case Expr.Applied(func, arg) => printpair("", func, " ", arg)
    case Expr.BinaryOp(left, op, right) => printpair("", left, " " + op.name + " ", right)
  }

  inPrecedence(exprPrinted, outer, outside)
}

cmd32.sc:36: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
                            ^
cmd32.sc:36: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
                                               ^
cmd32.sc:36: The outer reference in this type test cannot be checked at run time.
    case Expr.NaturalLiteral(_) | Expr.Variable(_, _) | Expr.Builtin(_) => (0, List())  // Never need parentheses.
                                                                    ^
cmd32.sc:37: The outer reference in this type test cannot be checked at run time.
    case Expr.Lambda(name, tipe, body) => (50, List(50, 50))
                    ^
cmd32.sc:38: The outer reference in this type test cannot be checked at run time.
    case Expr

defined function precedence
defined function inPrecedence
defined function prettyprint

In [33]:
// Test the pretty-printer.
Seq(
        ( Test1.test0, "1 + 2 + 3"),
        ( Test1.test1, "(n + 123) : Natural"),
        ( Test1.test2, "λ(n : Natural) → n + 1"),
        ( Test1.test3, "∀(n : Natural) → Natural"),
        ( Test1.test4, "(λ(n : Natural) → n + 1) : ∀(n : Natural) → Natural"),
        ( Test1.test5, "(λ(n : Natural) → n + 1) ((n + 123) : Natural)"),
        ( Test1.test6, "f (g x)"),
        
).zipWithIndex.foreach { case ((expr, expected), i) => expect(i >= 0 && prettyprint(expr) == expected) }